In [1]:
# https://vkvideo.ru/playlist/-202163717_4/video-202163717_456239939
# similar conversion dataset: https://github.com/30lm32/ml-ab-testing/blob/master/model.ipynb

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [13]:
df = pd.read_csv('ab_data.csv')
df

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


In [15]:
metrics = df.groupby('group', as_index=False).agg({'user_id': 'count',
                                                   'converted': 'sum'})
metrics['conversion'] = 100*metrics['converted']/metrics['user_id']
metrics

,group,user_id,converted,conversion
0,control,147202,17723,12.039918
1,treatment,147276,17514,11.891958


### Проверка различий конверсии

In [16]:
import statsmodels.stats.proportion as proportion

In [18]:
chi2stat, pval, table = proportion.proportions_chisquare(metrics['converted'],
                                                         metrics['user_id'])

In [19]:
pval

0.21611613269757673

### Мощность

А были у нас шансы вообще увидеть статистическую разницу \
Низкая мощность – больше шанс, что разницы не существует \
80% мощность означает, что в 80 случаев из 100 мы сможем обнаружить разницу

In [20]:
import statsmodels.stats.power as smp

In [22]:
chipower = smp.GofChisquarePower()

conversion_control = metrics['conversion'].values[0]/100
conversion_test = metrics['conversion'].values[1]/100
nobs = min(metrics['user_id'])

def chi2_effect_size(p0, p1):
    return np.sqrt(((p0 - p1)**2/p0))
    
chipower.solve_power(effect_size = chi2_effect_size(conversion_control,
                                                   conversion_test),
                     nobs= nobs,
                     alpha = 0.05,
                     power = None)  # было мало шансов увидеть стат значимую разницу

0.37315286694134825

### Минимальный обнаруживаемый эффект

Наименьшая разница между группами, которую мы можем разглядеть с желаемой уверенностью

Если мы говорим, что стат. значимой разницы нет, то это не означает, что разницы вовсе нет

Правильнее говорить, если разница и есть, то она не может быть обнаружена на рассматриваемом объеме данных

#### Вычисление кол-ва наблюдений, необходимых для получаемого уровня мощности

In [28]:
size_of_each_test_group = chipower.\
solve_power(effect_size= chi2_effect_size(conversion_control,
                                           conversion_test),
             nobs = None,
             alpha=0.05,
             power=0.8
            )
int(size_of_each_test_group*2)

863319

### Bootstrap

In [ ]:
import bootstrapped.bootstrap as bs 
import bootstrapped.stats_functions as bs_stats
import bootstrapped.compare_functions as bs_compare

test = df[df['test group'] == 'ad']['total ads'].values
ctrl = df[df['test group'] == 'psa']['total ads'].values
boot_results = bs.bootstrap_ab(test, ctrl,
                               stat_func = bs_stats.mean,
                              compare_func = bs_compare.difference,
                              return_distribution=True)
# получаем массив разниц
# если ноль входит в ДИ, то стат. разница не обнаружена

https://vkteam.medium.com/practitioners-guide-to-statistical-tests-ed2d580ef04f

CTR – clickthrough rate, number of clicks

$M=\frac{1}{|Users|}\sum_{u \in Users} clicks_u$

АБ-Тесты могут быть характеризованы: 
* FPR – вероятность отвергнуть H0 при истинной H0
* Sensitivity – вероятность отвергнуть H0 при ошибочной H0

FPR=0 если будем всегда принимать H0
Sensitivity = 100 если всегда будем отвергать H0

Trade-off между FPR и Sensitivity будет ROC кривая
Идеально когда низкий FPR и высокий Sensitivity

Распределение просмотров рекламы является лог-нормальное распределение
Распределение CTR обычно является бета-распределение

### Practicum Yandex

In [103]:
p1 = 0.0512
p2 =  0.0526
mde = 0.000765
n1 = 1321882
n2 = 1322630
ese = np.sqrt((p1*(1-p1)/n1) + (p2*(1-p2)/n2))
z_prop = (p2 - p1 - mde)/ese
z_prop

2.3276017578560886

In [110]:
import scipy
perc = 1.5/100
mde = 0.051*perc
var = 0.048
n = 2645900
zb = np.sqrt(n/(2*var))*mde + scipy.stats.norm.ppf(0.05/2)  # z-значение
scipy.stats.norm.cdf(zb)  # мощность при расч. MDE и n

0.9801188664699144

### Pandas